In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'domain': 'spouse',
    'debug': False,
    'postgres': False,
    'max_explanations': 30,
    'seed': 1234567,
    'parallelism': 1,
    'splits': [0,1,2],
    'disc_model_class': 'logreg',
    'supervision': 'majority',
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_spouse.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

if config['debug']:
    print("NOTE: --debug=True: modifying parameters...")
    config['max_docs'] = 100
    config['gen_model_search_space'] = 2
    config['disc_model_search_space'] = 2
    config['gen_params_default']['epochs'] = 25
    config['disc_params_default']['n_epochs'] = 5

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

Overwriting domain=None to domain=spouse
Overwriting print_freq=1 to print_freq=5
Overwriting LF_acc_prior_weight_default=1.0 to LF_acc_prior_weight_default=0.5
Overwriting decay=0.95 to decay=0.99
Overwriting init_class_prior=0 to init_class_prior=-1.15
Overwriting reg_param=0.1 to reg_param=0.5
Overwriting seed=0 to seed=1234567
Overwriting disc_model_class=lstm to disc_model_class=logreg
Overwriting supervision=generative to supervision=majority
Overwriting max_explanations=None to max_explanations=30


In [5]:
from snorkel.models import Document

session.query(Document).count()

2591

In [6]:
for split in [0,1,2]:
    print(session.query(pipe.candidate_class).filter(
        pipe.candidate_class.split == split).count())

22195
2796
2697


In [7]:
# %time pipe.parse()

In [8]:
# %time pipe.extract()

In [9]:
# %time pipe.load_gold()

In [10]:
# %time pipe.featurize()

In [11]:
%time pipe.collect()

Read 30 explanations from /Users/bradenjh/repo...ta/gradturk_explanations30.tsv
Linking candidates...
# CANDIDATES: 2796
Building list of target candidate ids...
Collected 30 unique target candidate ids from 30 explanations.
Gathering desired candidates...
Found 30/30 desired candidates
Linking explanations to candidates...
Linked 30/30 explanations
Since max_explanations > len(explanations), using all 30 Explanations
Calling babbler...
Created grammar with 596 rules
Flushing all parses from previous explanation set.
27 explanation(s) out of 30 were parseable.
143 parse(s) generated from 30 explanation(s).
39 parse(s) remain (104 parse(s) removed by DuplicateSemanticsFilter).
17 parse(s) remain (22 parse(s) removed by ConsistencyFilter).
### Applying labeling functions to split 1
[========================================] 100%

### Done in 6.9s.

17 parse(s) remain (0 parse(s) removed by UniformSignatureFilter: (0 None, 0 All)).
16 parse(s) remain (1 parse(s) removed by DuplicateSignat

In [15]:
pipe.babbler.get_label_matrix(split=1)

Clearing existing...
Running UDF...
[=                                       ] 2%

[autoreload of pipeline_module failed: Traceback (most recent call last):
  File "/Users/bradenjh/anaconda3/envs/snorkel/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named pipeline_module
]


[========================================] 100%



<2796x2 sparse matrix of type '<type 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
# %time pipe.label()

In [ ]:
# %time pipe.supervise()